In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [33]:
filter_df = vine_df.filter(vine_df.total_votes >= 20)
filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [34]:
vote_filter_df = filter_df.filter(filter_df.helpful_votes/filter_df.total_votes >= 0.5)
vote_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [35]:
paid_votes_df = vote_filter_df.filter(vote_filter_df.vine == 'Y')
paid_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

In [36]:
unpaid_votes_df = vote_filter_df.filter(vote_filter_df.vine == 'N')
unpaid_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [56]:
# Total number of paid reviews
paid_reviews_total = paid_votes_df.agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "total_votes")
paid_reviews_total.show()

+-----------+
|total_votes|
+-----------+
|         60|
+-----------+



In [57]:
# Total number of unpaid reviews
unpaid_reviews_total = unpaid_votes_df.agg({"total_votes": "count"}).withColumnRenamed("count(total_votes)", "total_votes")
unpaid_reviews_total.show()

+-----------+
|total_votes|
+-----------+
|      14477|
+-----------+



In [61]:
# Total five star reviews paid
paid_5star_reviews_total = paid_votes_df.filter(paid_votes_df.star_rating == 5).agg({"star_rating": "count"}).withColumnRenamed("count(star_rating)", "star_ratings")
paid_5star_reviews_total.show()

+------------+
|star_ratings|
+------------+
|          34|
+------------+



In [62]:
# Total five star reviews unpaid
unpaid_5star_reviews_total = unpaid_votes_df.filter(unpaid_votes_df.star_rating == 5).agg({"star_rating": "count"}).withColumnRenamed("count(star_rating)", "star_ratings")
unpaid_5star_reviews_total.show()

+------------+
|star_ratings|
+------------+
|        8212|
+------------+



In [72]:
# percentage of paid five star reviews
paid_percent = 34/60
paid_percent

0.5666666666666667

In [73]:
# percentage of unpaid five star reviews
unpaid_percent = 8212/14477
unpaid_percent

0.5672445948746287